<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Relatedness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.2 MB/s eta 0:00:00


In [5]:
!pip install transformers

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [8]:
#%run '/content/drive/MyDrive/ColabNotebooks/Main.ipynb'

In [9]:
from transformers import LongformerTokenizer, LongformerModel
import torch

def calculate_relatedness_with_longgormer():
  # Initialize the Longformer tokenizer and model
  model_name = "allenai/longformer-base-4096"
  tokenizer = LongformerTokenizer.from_pretrained(model_name)
  model = LongformerModel.from_pretrained(model_name)
  # List to store encoded data
  encoded_data = []

  # Your dataset of text, e.g., comments, replies, submissions
  text_data = []
  for comment in flat_comments_list:
      text_data.append(comment['body'])

  for comment in flat_comments_list:
      # Tokenize and encode the text
      encoded = tokenizer.encode(comment['body'], add_special_tokens=True, max_length=4096, truncation=True, return_tensors="pt")

      # Append the encoded text to the list
      # encoded_data.append(input_ids)
      comment['body_encoded'] = encoded
  # inputs_embeddings = []
  counter = 0
  for comment in flat_comments_list:
    # Convert input_ids to a PyTorch tensor
    # Modify this line to create a tensor using .clone().detach()
    data = comment['body_encoded'].clone().detach()
    # Forward pass through BERT
    with torch.no_grad():
        outputs = model(data) #.unsqueeze(0))  Add batch dimension (1) for single input

    # Extract embeddings for the [CLS] token (the first token in the sequence)
    comment['body_embeddings'] = outputs[0][:, 0, :].numpy()
    counter += 1
    print(counter)

  # Initialize the Longformer tokenizer and model
  model_name = "allenai/longformer-base-4096"
  tokenizer = LongformerTokenizer.from_pretrained(model_name)
  model = LongformerModel.from_pretrained(model_name)

  # List to store encoded data and corresponding comments
  encoded_data = []
  comment_list = []
  batch_size = 16  # You can adjust this based on your hardware capabilities

  counter = 0
  # Process comments in batches
  for i in range(0, len(flat_comments_list), batch_size):
      print(counter)
      counter+= 1
      batch_comments = flat_comments_list[i:i + batch_size]
      batch_texts = [comment['body'] for comment in batch_comments]

      # Tokenize and encode the text for the batch
      batch_encoded = tokenizer(batch_texts, add_special_tokens=True, max_length=4096, truncation=True, return_tensors="pt", padding=True)

      with torch.no_grad():
          # Forward pass through BERT for the batch
          outputs = model(batch_encoded['input_ids'])

      # Extract embeddings for the [CLS] token (the first token in the sequence) for the batch
      batch_embeddings = outputs[0][:, 0, :].numpy()

      # Store the encoded data and corresponding comments
      encoded_data.extend(batch_embeddings)
      comment_list.extend(batch_comments)

  # Now, 'encoded_data' contains the embeddings, and 'comment_list' contains the corresponding comments

In [10]:
!pip install datsets transformers[sentencepiece]

!pip install sentencepiece


ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.3 MB/s eta 0:00:00


In [11]:
from transformers import T5Tokenizer, T5Model
import torch

def google_longformer():
  # Initialize the LongT5 tokenizer and model
  model_name = "google/long-t5-tglobal-base"
  tokenizer = T5Tokenizer.from_pretrained(model_name)
  model = T5Model.from_pretrained(model_name)

  # List to store encoded data and corresponding comments
  encoded_data = []

  for comment in flat_comments_list:
      # Tokenize and encode the text
      encoded = tokenizer.encode(comment['body'], add_special_tokens=True, max_length=4096, truncation=True, return_tensors="pt")

      # Forward pass through the LongT5 model
      with torch.no_grad():
          outputs = model(input_ids=encoded)

      # Extract embeddings for the [CLS] token (the first token in the sequence)
      comment['body_embeddings'] = outputs.last_hidden_state[:, 0, :].numpy()

      # Append the encoded data
      encoded_data.append(comment)

  # Now, 'encoded_data' contains the embeddings for your comments


In [12]:
def jaccard_similarity(comment1, comment2):
    set1 = set(comment1.lower().split())  # Convert to lowercase and split into words
    set2 = set(comment2.lower().split())
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    return intersection / union



In [13]:
from transformers import LongformerTokenizer, LongformerModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def longformer_sample():
  # Sample comments and their IDs
  sample_comment = {
      'id': 'iqmwd8f',
      'body': "I'm tired of this life. what should I do?",
      'author': 't2_57iup',
      'reply_to': 'xsyg0d',
      'conversation_id': 'xsyg0d'
  }

  sample_reply_to_comment = {
      'id': 'xsyg0d',
      'body': "I'm so good at playing guitar",
      'author': 't2_57iup',
      'reply_to': None,
      'conversation_id': 'xsyg0d'
  }

  # Initialize the Longformer tokenizer and model
  model_name = "allenai/longformer-base-4096"
  tokenizer = LongformerTokenizer.from_pretrained(model_name)
  model = LongformerModel.from_pretrained(model_name)

  # Tokenize and encode the sample comment and its reply
  sample_comment_encoded = tokenizer.encode(sample_comment['body'], add_special_tokens=True, max_length=4096, truncation=True, return_tensors="pt")
  sample_reply_encoded = tokenizer.encode(sample_reply_to_comment['body'], add_special_tokens=True, max_length=4096, truncation=True, return_tensors="pt")

  # Forward pass through the Longformer model for the sample comment and reply
  with torch.no_grad():
      comment_output = model(sample_comment_encoded)
      reply_output = model(sample_reply_encoded)

  # Extract embeddings for the [CLS] token (the first token in the sequence) for the sample comment and reply
  comment_embedding = comment_output.last_hidden_state[:, 0, :].numpy()[0]
  reply_embedding = reply_output.last_hidden_state[:, 0, :].numpy()[0]

  print(comment_embedding)
  # Calculate the cosine similarity between the sample comment and its reply
  similarity = cosine_similarity([comment_embedding], [reply_embedding])

  # Print the similarity and both texts
  print("Cosine Similarity:", similarity)
  print("j similarity:", jaccard_similarity(sample_comment['body'], sample_reply_to_comment['body']))
  print("Comment:", sample_comment['body'])
  print("Reply:", sample_reply_to_comment['body'])

In [46]:
import pickle

# Reading from a file using Pickle
with open(f"{base_path}conversations.pkl", 'rb') as file:
    conversations = pickle.load(file)

# Print the loaded variables
print("Len conversations:", len(conversations))

Len conversations: 33493


In [48]:
for id in conversations:
  id_index_map = dict()
  comments = conversations[id]['comments']
  selected_comments = []
  for comment in comments:
    if not(comment['body'] == '[deleted]' or comment['body'].endswith('[removed]')):
        id_index_map[comment['id']] = len(selected_comments)
        selected_comments.append(comment['body'])
  # Create a TF-IDF vectorizer
  tfidf_vectorizer = TfidfVectorizer()

  # Fit and transform the comments to obtain TF-IDF vectors
  #print(conversations[id]['full_text'])
  tfidf_matrix = tfidf_vectorizer.fit_transform(selected_comments)

  # Calculate the cosine similarity between comments
  cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

  for pair in conversations[id]['pairs']:
    pair['relatedness_score'] = cosine_sim[id_index_map[pair['comment_id']]][id_index_map[pair['reply_id']]]
  # Print the cosine similarity matrix
  #print("Cosine Similarity Matrix:")
  #print(cosine_sim)

In [52]:
import pickle

# Writing to a file using Pickle
with open(f"{base_path}conversations_with_relatedness.pkl", 'wb') as file:
    pickle.dump(conversations, file)

print("Variables written to the file.")

Variables written to the file.


In [35]:
# Create a dictionary
my_dict = {'a': 1, 'b': 2, 'c': 3}

# Get a list of values
values_list = list(my_dict.values())

# Print the list of values
print(values_list)


[1, 2, 3]


In [24]:
len(flat_comments)

542128

In [25]:
flat_comments[0]

{'comment': 'MLB Sat Info\n\nPlay\xa0OnAll underdogs with a money line of +100 or higher\xa0(SEATTLE)team with a terrible SLG (&lt;=.400) against a very good starting pitcher (WHIP&lt;=1.300) -AL, with a very good bullpen whose WHIP is 1.250 or better on the season217-224\xa0over the last 5 seasons.49.2%\xa0(89.1\xa0units)',
 'reply': 'Play\xa0OnAll underdogs with a money line of +100 or higher\xa0(TAMPA BAY)team with a terrible SLG (<=.400) against a very good starting pitcher (WHIP<=1.300) -AL, with a very good bullpen whose WHIP is 1.250 or better on the season217-224\xa0over the last 5 seasons.49.2%\xa0(89.1\xa0units)',
 'comment_id': 'xyt38s',
 'reply_id': 'iromk5a',
 'global_id': 'xyt38s_iromk5a'}

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Sample comments
comments = [
    "I've been out looking for Ghoulia today, good to know why I couldn't find her.",
    "Ghoulia is an elusive character, isn't she? Maybe she's hiding from the monsters.",
    "I went for a walk in the park today.",
]

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the comments to obtain TF-IDF vectors
tfidf_matrix = tfidf_vectorizer.fit_transform(comments)

# Calculate the cosine similarity between comments
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Print the cosine similarity matrix
print("Cosine Similarity Matrix:")
print(cosine_sim)


Cosine Similarity Matrix:
[[1.         0.04307671 0.13535702]
 [0.04307671 1.         0.06419055]
 [0.13535702 0.06419055 1.        ]]
